In [1]:
import glob
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, ConcatDataset
import os
import re
from shutil import copy2
import torch.utils.data
from torch import nn, optim
import numpy as np
from itertools import product
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from skimage.color import lab2rgb
from skimage.color import rgb2lab, rgb2gray
from torchvision import datasets
import warnings
warnings.filterwarnings("ignore")

def data_load():
    image_list = glob.glob('face_images/*.jpg')
    print("Length of given Image List", len(image_list))
    train_test_split()
    training_image_list = glob.glob('data/train/class/*.jpg')
    test_image_list = glob.glob('data/test/class/*.jpg')
    print("Length of training Image List", len(training_image_list))
    print("Length of testing Image List", len(test_image_list))
    
def train_test_split():
        os.makedirs('data/train/class/', exist_ok=True)
        os.makedirs('data/test/class/', exist_ok=True)
        os.makedirs('Model/Colorizer/', exist_ok=True)
        os.makedirs('Model/Regressor/', exist_ok=True)
        os.makedirs('Plots/Colorizer/', exist_ok=True)
        os.makedirs('outputs_sigmoid/gray/',exist_ok=True)
        os.makedirs('outputs_sigmoid/color/',exist_ok=True)
        os.makedirs('outputs_sigmoid/disp/',exist_ok=True)
        os.makedirs('outputs_tanh/gray/',exist_ok=True)
        os.makedirs('outputs_tanh/color/',exist_ok=True)


        number_of_images = len(next(os.walk('face_images'))[2])
        print("Number of images - ", number_of_images)

        for i, file in enumerate(os.listdir('face_images')):
            if i < (0.1 * number_of_images):
                copy2('face_images/' + file, 'data/test/class/' + file)
                continue
            else: 
                copy2('face_images/' + file, 'data/train/class/' + file)

        print("Training Set Size : ", len(next(os.walk('data/train/class'))[2]))
        print("Test Set Size : ", len(next(os.walk('data/test/class'))[2]))
def build_dataset(cuda=False, num_workers=1,
                  activation_function="sigmoid"):
    transform = transforms.Compose([
        transforms.Resize(128),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(128)
    ])

    train_datasets = []
    if activation_function == "sigmoid" or activation_function == 'tanh':
        train_datasets.append(AugmentImageDataset('data/train'))
    elif activation_function == "relu":
        train_datasets.append(AugmentImageDataset_RELU('data/train'))

    for i in range(9):
        if activation_function == "sigmoid":
            train_datasets.append(AugmentImageDataset('data/train', transform))
        elif activation_function == "relu":
            train_datasets.append(AugmentImageDataset_RELU('data/train', transform))
        elif activation_function == "tanh":
            train_datasets.append(AugmentImageDataset_Tanh('data/train',
                                                                        transform))

    augmented_dataset = ConcatDataset(train_datasets)
    print("Length of Augmented Dataset", len(augmented_dataset))

    train_loader_args = dict(shuffle=True,
                             batch_size=16,
                             num_workers=num_workers, pin_memory=True) \
        if cuda else dict(shuffle=True, batch_size=32)

    augmented_dataset_batch_train = DataLoader(dataset=augmented_dataset, **train_loader_args)
    augmented_dataset_batch_test = DataLoader(dataset=AugmentImageDataset('data/test'))

    return augmented_dataset_batch_train, augmented_dataset_batch_test

def execution_colorizer_sigmoid():
    activation_function = "sigmoid"
    save_path = {'grayscale': 'outputs_sigmoid/gray/', 'colorized': 'outputs_sigmoid/color/'}
    device, is_cuda_present, num_workers = get_device()
    model_name = "Model/Colorizer/Colorizer_sigmoid_epoch_{0}_lr_{1}_weight_decay_{2}.pth"

    print("Device: {0}".format(device))
    augmented_dataset_batch_train, \
    augmented_dataset_batch_test = build_dataset(is_cuda_present, num_workers,activation_function)

    colorizer = Manage_Colorize()
    colorizer.train(augmented_dataset_batch_train,activation_function, device)
    colorizer.test(augmented_dataset_batch_test, activation_function,save_path, device)
    train_regressor(augmented_dataset_batch_train, device)
    test_regressor(augmented_dataset_batch_test, device)
def execution_colorizer_tanh():
    activation_function = "tanh"
    save_path = {'grayscale': 'outputs_tanh/gray/', 'colorized': 'outputs_tanh/color/'}
    device, is_cuda_present, num_workers = get_device()
    model_name = "Model/Colorizer/Colorizer_tanh_epoch_{0}_lr_{1}_weight_decay_{2}.pth"

    print("Device: {0}".format(device))
    augmented_dataset_batch_train, \
    augmented_dataset_batch_test = build_dataset(is_cuda_present, num_workers,activation_function)
    colorizer = Manage_Colorize()
    colorizer.train(augmented_dataset_batch_train,activation_function, device)
    colorizer.test(augmented_dataset_batch_test, activation_function,save_path, device)
    train_regressor(augmented_dataset_batch_train, device)
    test_regressor(augmented_dataset_batch_test, device)
    
def train_regressor(augmented_dataset_batch_train, device):
    data_loader = augmented_dataset_batch_train
    saved_model_path = "Model/Regressor/Regressor.pth"
    epochs = 400
    lr = 0.0001
    weight_decay = 1e-5
    in_channel = 1
    hidden_channel = 3
    out_dims = 2
    loss_plot_path = "Model/Regressor/Regressor_Loss_plot.jpeg"   

    print("..Regressor training started..")
    model = Regressor(in_channel=in_channel,
                        hidden_channel=hidden_channel,
                        out_dims=out_dims,
                        train_mode="regressor").to(device)

    lossF = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)
    loss_train = []
    for epoch in range(epochs):
        total_loss = 0
        model.train()

        for batch in data_loader:
            l_channel, a_channel, b_channel = batch
            l_channel = l_channel.to(device)

            a_b_mean = get_ab_mean(a_channel, b_channel)
            a_b_mean_hat = model(l_channel)

            if torch.cuda.is_available():
                loss = lossF(a_b_mean_hat.float().cuda(),
                                 a_b_mean.float().cuda()).to(device)
            else:
                loss = lossF(a_b_mean_hat.float(),
                                a_b_mean.float()).to(device)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print("epoch: {0}, loss: {1}"
                .format(epoch, total_loss))
        loss_train.append(total_loss)

    plot_loss_epoch(loss_train, loss_plot_path)
    torch.save(model.state_dict(), saved_model_path)

def test_regressor(augmented_dataset_batch_test, device):
    data_loader = augmented_dataset_batch_test
    saved_model_path = "Model/Regressor/Regressor.pth"
    in_channel = 1
    hidden_channel = 3
    out_dims = 2

    print("..Regressor testing started..")

    model = Regressor(in_channel=in_channel,
                          hidden_channel=hidden_channel,
                          out_dims=out_dims,
                          train_mode="regressor").to(device)
    model.load_state_dict(torch.load(saved_model_path, map_location=device))

    a_list = []
    b_list = []
    lossF = nn.MSELoss()
    total_loss = 0
    loss_train = []
    for batch in data_loader:
        l_channel, a_channel, b_channel = batch
        l_channel = l_channel.to(device)

        a_b_mean = get_ab_mean(a_channel, b_channel)
        a_b_mean_hat = model(l_channel).detach()

        if torch.cuda.is_available():
            loss = lossF(a_b_mean_hat.float().cuda(),
                             a_b_mean.float().cuda()).to(device)
        else:
            loss = lossF(a_b_mean_hat.float(),
                             a_b_mean.float()).to(device)

        loss_train.append(loss.item())

        a_b_pred = a_b_mean_hat[0].cpu().numpy()
        a_list.append(a_b_pred[0])
        b_list.append(a_b_pred[1])

    print("MSE:", np.average(np.asarray(loss_train)))
    print("Image_num || Mean a || Mean b")
    for i in range(1, len(a_list)):
        print("Image: {0} mean_a: {1} mean_b:{2}".format(
            i, (a_list[i] * 255) - 128, (b_list[i] * 255) - 128
        ))
    
def get_device():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    is_cuda_present = True if torch.cuda.is_available() else False
    num_workers = 8 if is_cuda_present else 0
    return device, is_cuda_present, num_workers

class AugmentImageDataset(datasets.ImageFolder):
    def __getitem__(self, index):
        global img_a, img_b, img_gray
        path, target = self.imgs[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        img_lab = rgb2lab(np.asarray(img))
        img_lab = img_lab + 128
        img_lab = img_lab / 255
        img_a = torch.from_numpy(img_lab[:, :, 1:2].transpose((2, 0, 1))).float() 
        img_b = torch.from_numpy(img_lab[:, :, 2:3].transpose((2, 0, 1))).float()
        img_gray = torch.from_numpy(rgb2gray(np.asarray(img))).unsqueeze(0).float()
        return img_gray, img_a, img_b

class AugmentImageDataset_Tanh(datasets.ImageFolder):
    def __getitem__(self, index):
        global img_a, img_b, img_gray
        path, target = self.imgs[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        img_lab = rgb2lab(np.asarray(img))
        img_lab = img_lab + 128
        img_lab = img_lab / 255
        img_a = torch.from_numpy(img_lab[:, :, 1:2].transpose((2, 0, 1))).float()
        img_b = torch.from_numpy(img_lab[:, :, 2:3].transpose((2, 0, 1))).float()
        img_gray = torch.from_numpy(rgb2gray(np.asarray(img))).unsqueeze(0).float()
        return img_gray, img_a, img_b

class AugmentImageDataset_RELU(datasets.ImageFolder):
    def __getitem__(self, index):
        global img_a, img_b, img_gray
        path, target = self.imgs[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        img_lab = rgb2lab(np.asarray(img))
        img_a = torch.from_numpy(img_lab[:, :, 1:2].transpose((2, 0, 1))).float() 
        img_b = torch.from_numpy(img_lab[:, :, 2:3].transpose((2, 0, 1))).float()
        img_gray = torch.from_numpy(rgb2gray(np.asarray(img))).unsqueeze(0).float()

        return img_gray, img_a, img_b
    
def get_ab_mean(a_channel, b_channel):
    a_channel_mean = a_channel.mean(dim=(2, 3))
    b_channel_mean = b_channel.mean(dim=(2, 3))
    a_b_mean = torch.cat([a_channel_mean,
                              b_channel_mean], dim=1)
    return a_b_mean

def plot_loss_epoch(train_loss_avg, fig_name):
    plt.ion()
    fig = plt.figure()
    plt.plot(train_loss_avg)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.draw()
    plt.savefig(fig_name, dpi=220)
    plt.clf()

class Regressor(nn.Module):
    def __init__(self, in_channel=1, hidden_channel=3, out_dims=2,
                 train_mode="regressor"):
        super(Regressor, self).__init__()
        self.train_mode = train_mode

        self.feature_maps = nn.Sequential(
            nn.Conv2d(in_channels=in_channel, out_channels=32,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=32, out_channels=64,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=64, out_channels=128,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=128, out_channels=256,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=256, out_channels=256,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=256, out_channels=512,
                      kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(),
        )

        if self.train_mode == "regressor":
            self.lin = nn.Linear(in_features=512 * 2 * 2, out_features=out_dims)

    def forward(self, x):
        feature_maps = self.feature_maps(x)
        if self.train_mode == "regressor":
            y_hat = torch.sigmoid(self.lin(feature_maps.reshape(-1, 512 * 2 * 2)))
            return y_hat

        else:
            return feature_maps

class Colorizer(nn.Module):
    def __init__(self, in_channel=3, hidden_channel=3, out_channel=2,
                 activation_function="sigmoid"):
        super(Colorizer, self).__init__()
        self.activation_function = activation_function
        self.feature_maps = Regressor(in_channel=1, hidden_channel=3, out_dims=2,
                                      train_mode="colorizer")
        self.up_sample = nn.Sequential(
            nn.ConvTranspose2d(in_channels=512, out_channels=256,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(in_channels=256, out_channels=256,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(in_channels=256, out_channels=128,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(in_channels=128, out_channels=64,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(in_channels=64, out_channels=32,
                               kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(in_channels=32, out_channels=out_channel,
                               kernel_size=4, stride=2, padding=1)
        )

    def forward(self, x):
        if self.activation_function == "sigmoid":
            return torch.sigmoid(self.up_sample(self.feature_maps(x)))
        elif self.activation_function == "tanh":
            return torch.tanh(self.up_sample(self.feature_maps(x)))
        elif self.activation_function == "relu":
            return torch.relu(self.up_sample(self.feature_maps(x)))

def show_img(image):
    plt.figure(figsize=(20, 20))
    np_img = image.numpy()
    plt.imshow(np.transpose(np_img, (1, 2, 0)))
    plt.show()
def to_rgb(grayscale_input, ab_input, activation_function="tanh",
               save_path=None, save_name=None, device="cpu"):
    plt.clf()
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy()  # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None:
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))
def show_img_tensor(image):
    plt.figure(figsize=(20, 20))
    plt.imshow(image.permute(1, 2, 0))
    plt.show()
    plt.clf()
def show_output_image(gray, orig, recons, fig_name):
    plt.clf()
    f = plt.figure()
    f.add_subplot(1, 3, 1)
    plt.imshow(mpimg.imread(gray))
    plt.axis('off')
    f.add_subplot(1, 3, 2)
    plt.imshow(mpimg.imread(orig))
    plt.axis('off')
    f.add_subplot(1, 3, 3)
    plt.imshow(mpimg.imread(recons))
    plt.axis('off')
    plt.draw()
    plt.savefig(fig_name, dpi=220)
    plt.clf()
    plt.close()
    
class EarlyStopping_DCN:

    def __init__(self, patience=7, verbose=False, delta=0,
                 model_path=None,
                 trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.model_path = model_path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            self.trace_func(
                f'Validation loss decreased ({self.val_loss_min} --> {val_loss}).  Saving model ...')
        torch.save(model.state_dict(), self.model_path)
        self.val_loss_min = val_loss
        
class Manage_Colorize:
    def train(self,augmented_dataset_batch_train, activation_function, device):
        print("Activation Function: ", activation_function)

        train_data_loader = augmented_dataset_batch_train
        saved_model_path ="Model/Colorizer/Colorizer_{0}.pth".format(activation_function)

        epochs = 400
        lr = .0001
        weight_decay = 1e-5
        in_channel = 3
        hidden_channel = 3
        loss_plot_path = "Model/Colorizer/Colorizer_Loss_plot_{0}.jpeg".format(activation_function)

        print("..Colorizer Training started..")
        model = Colorizer(in_channel=3, hidden_channel=3,
                          activation_function=activation_function).to(device)

        lossF = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=lr,
                               weight_decay=weight_decay)
        loss_train = []
        early_stopping = EarlyStopping_DCN(patience=50, verbose=True,
                                           model_path=saved_model_path)
        for epoch in range(epochs):
            total_loss_train = 0
            total_loss_val = 0
            model.train()

            for batch in train_data_loader:
                l_channel, a_channel, b_channel = batch
                l_channel = l_channel.to(device)

                a_b_channel = torch.cat([a_channel, b_channel], dim=1)
                a_b_channel_hat = model(l_channel)

                if torch.cuda.is_available():
                    loss = lossF(a_b_channel_hat.float().cuda(),
                                 a_b_channel.float().cuda()).to(device)
                else:
                    loss = lossF(a_b_channel_hat.float(),
                                 a_b_channel.float()).to(device)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss_train += loss.item()

            print("epoch: {0}, loss: {1}"
                  .format(epoch, total_loss_train))
            loss_train.append(total_loss_train)

            if early_stopping.early_stop:
                print("Early stopping")
                break

        plot_loss_epoch(loss_train, loss_plot_path)
        torch.save(model.state_dict(), saved_model_path)

    @staticmethod
    def validate(model, val_data_loader, lossF, device):
        loss_valid = []
        model.eval()

        # val treated
        for batch in val_data_loader:
            l_channel, a_channel, b_channel = batch
            l_channel = l_channel.to(device)

            a_b_channel = torch.cat([a_channel, b_channel], dim=1)
            a_b_channel_hat = model(l_channel)

            if torch.cuda.is_available():
                loss = lossF(a_b_channel_hat.float().cuda(),
                             a_b_channel.float().cuda()).to(device)
            else:
                loss = lossF(a_b_channel_hat.float(),
                             a_b_channel.float()).to(device)

            loss_valid.append(loss.item())

        valid_loss = np.average(loss_valid)
        return valid_loss

    def test(self, augmented_dataset_batch_train, activation_function, save_path,device):
        print(activation_function)
        data_loader = augmented_dataset_batch_train
        saved_model_path ="Model/Colorizer/Colorizer_{0}.pth".format(activation_function)

        epoch = 400
        lr = .0001
        weight_decay = 1e-5
        in_channel = 3
        hidden_channel = 3
        loss_plot_path = "Model/Colorizer/Colorizer_Loss_plot_{0}.jpeg".format(activation_function)
        

        print("..Colorizer Test started..")
        model = Colorizer(in_channel=in_channel,
                          hidden_channel=hidden_channel,
                          activation_function=activation_function).to(device)
        model.load_state_dict(torch.load(saved_model_path, map_location=device))

        lossF = nn.MSELoss()
        serial_num = 0
        for batch in data_loader:
            serial_num += 1
            l_channel, a_channel, b_channel = batch
            l_channel = l_channel.to(device)

            a_b_channel = torch.cat([a_channel, b_channel], dim=1)
            a_b_channel_hat = model(l_channel).detach()

            if torch.cuda.is_available():
                loss = lossF(a_b_channel_hat.float().cuda(),
                             a_b_channel.float().cuda()).to(device)
            else:
                loss = lossF(a_b_channel_hat.float(),
                             a_b_channel.float()).to(device)

            print("Image: {0}, loss: {1}".format(serial_num, loss.item()))

            save_name_orig = 'Orig_img_epoch_{0}_lr_{1}_wt_decay{2}_serial_{3}_{4}.jpg' \
                .format(epoch, lr, weight_decay, serial_num,activation_function)
            save_name_recons = 'Recons_img_epoch_{0}_lr_{1}_wt_decay{2}_serial_{3}_{4}.jpg' \
                .format(epoch, lr, weight_decay, serial_num,activation_function)

            to_rgb(l_channel[0].cpu(), a_b_channel[0].cpu(),
                         activation_function,
                         save_path=save_path, save_name=save_name_orig, device=device)
            to_rgb(l_channel[0].cpu(), a_b_channel_hat[0].cpu(),
                         activation_function,
                         save_path=save_path, save_name=save_name_recons, device=device)

        self.show_final_image_grid(epoch, lr, weight_decay, save_path,activation_function)

    @staticmethod
    def show_final_image_grid(epoch, lr, weight_decay, save_path,activation_function):

        color_path = save_path['colorized']
        gray_path = save_path['grayscale']

        for image_index in range(7, 70, 7):
            title = "Plots/Colorizer/epoch_{0}_lr_{1}_wt_{2}_serial_{3}_{4}.jpeg".\
                format(epoch, lr, weight_decay, image_index,activation_function)

            save_name_orig = 'Orig_img_epoch_{0}_lr_{1}_wt_decay{2}_serial_{3}_{4}.jpg' \
                .format(epoch, lr, weight_decay, image_index,activation_function)
            save_name_recons = 'Recons_img_epoch_{0}_lr_{1}_wt_decay{2}_serial_{3}_{4}.jpg' \
                .format(epoch, lr, weight_decay, image_index,activation_function)
            show_output_image(gray_path + save_name_orig, color_path + save_name_orig,
                                    color_path + save_name_recons, title)
            
        
if __name__ == '__main__':
    data_load()

    #print("For Sigmoid Function")
    #execution_colorizer_sigmoid()

    print("For Tanh Function")
    execution_colorizer_tanh()



Length of given Image List 750
Number of images -  751
Training Set Size :  675
Test Set Size :  76
Length of training Image List 674
Length of testing Image List 76
For Tanh Function
Device: cuda:0
Length of Augmented Dataset 6740
Activation Function:  tanh
..Colorizer Training started..
epoch: 0, loss: 8.621428595739417
epoch: 1, loss: 0.5397621112060733
epoch: 2, loss: 0.35630962281720713
epoch: 3, loss: 0.288689888373483
epoch: 4, loss: 0.22759299777681008
epoch: 5, loss: 0.19920429721241817
epoch: 6, loss: 0.18773529079044238
epoch: 7, loss: 0.1766558014205657
epoch: 8, loss: 0.16603903219220228
epoch: 9, loss: 0.16340830575791188
epoch: 10, loss: 0.14910932462953497
epoch: 11, loss: 0.1418149634700967
epoch: 12, loss: 0.13657412499014754
epoch: 13, loss: 0.12891608047357295
epoch: 14, loss: 0.12473352342203725
epoch: 15, loss: 0.1229697744565783
epoch: 16, loss: 0.11733721422206145
epoch: 17, loss: 0.11714104948623572
epoch: 18, loss: 0.11586829379666597
epoch: 19, loss: 0.112299

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>